# Import modules

In [1]:
%matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from scipy.linalg import sqrtm, block_diag
from math import pi
from qutip import *
import sympy
import functools, operator
from collections import OrderedDict
import itertools

Using matplotlib backend: TkAgg


# Two transmon + bus 
Following Magesan's paper

## Undriven

In [2]:
def hamiltonian_eff_H0(w1,w2,J,alpha1,alpha2,N):
    """ 
    Two-transmon effective Hamiltonian with zero excitation in bus. 
    Eq. 2.12, Magesan
    w1, w2:  angular freq
    """
    b1 = destroy(N)
    b2 = destroy(N)
    
    I = qeye(N)

    # Eq. 2.21, Magesan
    H1 = w1*tensor(b1.dag()*b1, I) + alpha1/2.0*tensor(b1.dag()*b1.dag()*b1*b1, I)
    H2 = w2*tensor(I, b2.dag()*b2) + alpha2/2.0*tensor(I, b2.dag()*b2.dag()*b2*b2)
    H12 = J*(tensor(b1.dag(), b2) + tensor(b1, b2.dag())) 
    
    H_eff = H1 + H2 + H12
    
    return H_eff

def hamiltonian_eff_Hd(wd, Omega_X, Omega_Y, t, N):
    """ 
    CR drive Hamiltonian. Eq.2.14.
    All arguments are single number, not an array
    wd: angular freq
    """
    b1 = destroy(N)  # qubit 2 is target qubit
#     Hd = (Omega_X * np.cos(wd*t) + Omega_Y * np.sin(wd*t)) * tensor(qeye(N),b2 + b2.dag())
    Hd = (Omega_X * np.cos(wd*t) + Omega_Y * np.sin(wd*t)) * tensor(b1 + b1.dag(),qeye(N))
    
    return Hd

def ZZ_computed(J, alpha1, alpha2, Delta):
    """ Eq.4.5 in Magesan"""
    return -J**2*(1/(Delta + alpha1) + 1/(-Delta + alpha2)) * 2

In [3]:
# parameters for IBM qubit
w1, w2 = 2*pi*5, 2*pi*5.2 # in GHz
J = 0.005*2*pi
alpha1, alpha2 = -0.35*2*pi, -0.35*2*pi # in GHz
N=5

H_eff = hamiltonian_eff_H0(w1,w2,J,alpha1,alpha2,N)
eigen = H_eff.eigenstates()
ZZ = eigen[0][5] - eigen[0][1] - eigen[0][2]  # in GHz

sympy.SparseMatrix(np.round((H_eff)[:, 0:13],2))
print(f'ZZ = {ZZ*1e3/(2*pi):.3f} (MHz)')
print(f'ZZ_by_formula = {ZZ_computed(J,alpha1,alpha2,w1-w2)*1e3/(2*pi):.3f} (MHz)')

ZZ = 0.423 (MHz)
ZZ_by_formula = 0.424 (MHz)


In [4]:
eigen[0]

array([  0.        ,  31.41514163,  32.67334851,  60.63216698,
        63.14392431,  64.09114939,  87.64991143,  91.4194034 ,
        93.30269448,  94.56601264, 112.46851434, 117.49482602,
       120.3217872 , 122.83078145, 123.78669517, 145.14018436,
       148.90897676, 150.79083124, 152.06261183, 175.61160088,
       178.11596075, 179.08655243, 203.87692006, 205.15844344,
       229.96458224])

## CR Driven: two-level qubit case

In [203]:
w1, w2 = 2*pi*5, 2*pi*5.2 # in GHz
J = 0.01*2*pi
alpha1, alpha2 = -0.35*2*pi, -0.35*2*pi
N =2
Omega_X, Omega_Y = 0.02*2*pi,0*2*pi

b1 = destroy(N)
b2 = destroy(N)

HA = w2*(tensor(b1.dag()*b1, qeye(N)) + tensor(qeye(N), b2.dag()*b2))
R = lambda t: (-1j*HA*t).expm()

H0 = hamiltonian_eff_H0(w1,w2,J,alpha1,alpha2,N)
Hd = lambda t:hamiltonian_eff_Hd(w2, Omega_X, Omega_Y, t, N)

HR = lambda t: R(t).dag() * (H0 + Hd(t)) * R(t) - HA

In [204]:
t_list = np.linspace(0, 2, 401)  # in ns
HR_list = [HR(t) for t in t_list]
H_R = functools.reduce(operator.add, HR_list) / len(HR_list)

In [205]:
H_R.conj().trans()
H_R

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.        +0.j          0.        +0.j          0.06248237-0.00025391j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.06283185+0.j
   0.06248237-0.00025391j]
 [ 0.06248237+0.00025391j  0.06283185+0.j         -1.25663706+0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.06248237+0.00025391j  0.        +0.j
  -1.25663706+0.j        ]]

In [17]:
avg_list = []
for N in range(2,401):
    temp = (functools.reduce(operator.add,[HR_list[i] for i in range(N)]) / N)
    temp = np.real(temp[2,0])
    avg_list.append(temp)
    
plt.plot(avg_list)
# avg_list

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 3.]]

## Least action

In [206]:
eigen = H_R.eigenstates()
# np.linalg.norm(eigen[1][3])

In [207]:
eigen[1][0]

Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[ 0.04191927+0.j        ]
 [ 0.06799578+0.00027632j]
 [-0.84850975-0.00344813j]
 [-0.52308748-0.00425146j]]

In [209]:
# find X
X_array = np.column_stack( (eigen[1][0],eigen[1][1],eigen[1][2],eigen[1][3]))
X = Qobj(X_array)

# find X_BD and XP
A, B = X[0:2,0:2], X[2:4, 2:4]
X_BD_array = block_diag(A,B)
X_BD = Qobj(X_BD_array)
XP = X_BD * X_BD.conj().trans()

# find T
T = X * X_BD.conj().trans() * XP.sqrtm().inv()

# find H_R_BD
T.dims = H_R.dims
H_R_BD =  T.conj().trans() * H_R * T
H_R_BD

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-1.2597286 +0.00000000e+00j -0.00310116+1.26023328e-05j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j]
 [-0.00310116-1.26023328e-05j -1.26284711+0.00000000e+00j
   0.        +0.00000000e+00j  0.        +0.00000000e+00j]
 [ 0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.00621004+0.00000000e+00j  0.00310116-1.26023328e-05j]
 [ 0.        +0.00000000e+00j  0.        +0.00000000e+00j
   0.00310116+1.26023328e-05j  0.00309153+0.00000000e+00j]]

In [210]:
w1, w2 = 2*pi*5, 2*pi*5.2 # in GHz
J = 0.01*2*pi
alpha1, alpha2 = -0.35*2*pi, -0.35*2*pi
N =2
Omega_X, Omega_Y = 0.02*2*pi,0*2*pi

Delta=w1-w2
print(J * Omega_X /np.sqrt(Delta**2+ Omega_X**2)/2)
print( (Delta-np.sqrt(Delta**2+ Omega_X**2))/2)

0.0031260015268123323
-1.2597708391340494


## CR Driven: three-level case

In [86]:
# get block-diagonal full H_RWA Hamiltonina, then take computational subspace
# to fine Pauli coefficients: ZX,ZZ,ZI,IX,and IZ

# Main question: how to compute H_RWA. How can we ignore other than e^{-iwt} or e^{iwt}.

In [192]:
w1, w2 = 2*pi*5.2, 2*pi*5 # in GHz
J = 0.01*2*pi
alpha1, alpha2 = -0.35*2*pi, -0.35*2*pi
Omega_X, Omega_Y = 20*2*pi,20*2*pi
N =3

b1 = destroy(N)
b2 = destroy(N)

H0 = hamiltonian_eff_H0(w1,w2,J,alpha1,alpha2,N)

In [193]:
# digonalize H0: remove J
eigen = H0.eigenstates()
eigen[1].size

# find U
U_array = np.column_stack( [eigen[1][i] for i in range(eigen[1].size)])
U = Qobj(U_array)
U.dims = H0.dims

# digonalize H0
H0_D = U.conj().trans() * H0 * U
sympy.Matrix(H0_D)

# get new Hd and HR
wd = (eigen[0][5]-eigen[0][2]+eigen[0][1])/2
Hd = lambda t:hamiltonian_eff_Hd(wd, Omega_X, Omega_Y, t, N)
Hd_D = lambda t: U.conj().trans() * Hd(t) * U

# To rotating frame
HA = wd*(tensor(b1.dag()*b1, qeye(N)) + tensor(qeye(N), b2.dag()*b2))
R = lambda t: (-1j*HA*t).expm()
HR = lambda t: R(t).dag() * (H0_D + Hd_D(t)) * R(t) - HA

In [194]:
(eigen[0][5]-eigen[0][2]+eigen[0][1])/2
# eigen[0]

31.418074040177185

In [195]:
# average over 10 period
t_list = np.linspace(0, 2, 401)  # in ns
HR_list = [HR(t) for t in t_list]
H_R = functools.reduce(operator.add,HR_list) / len(HR_list)

In [196]:
HR_list[0]+HR_list[1]
H_R

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[ 0.00000000e+00+0.00000000e+00j -3.13793833e+00+3.12190493e+00j
  -3.22138515e-01+1.45703351e-03j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j]
 [-3.13793833e+00-3.12190493e+00j -5.28128198e-03+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  8.30506439e-03+0.00000000e+00j
   1.02539063e+01-1.02015137e+01j  3.18534313e-01-1.44073170e-03j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j]
 [-3.22138515e-01-1.45703351e-03j  0.00000000e+00+0.00000000e+00j
  -3.01604507e+01+0.00000000e+00j  1.27716875e-04+1.27064301e-04j
   4.52224368e-01+0.00000000e+00j -1.13805883e+01+1.13224388e+01j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j]
 [ 0.00

In [197]:
avg_list = []
for N in range(2,401):
    temp = (functools.reduce(operator.add,[HR_list[i] for i in range(N)]) / N)
    temp = np.real(temp[0,1])
    avg_list.append(temp)
    

In [93]:
# plt.plot(avg_list)
f=np.fft.fft(avg_list)
plt.plot(np.absolute(f))
plt.yscale('log')

In [198]:
# least action
eigen = H_R.eigenstates()

# find X
X_array = np.column_stack( [eigen[1][i] for i in range(eigen[1].size)])
X = Qobj(X_array)

# find X_BD and XP
A, B, C = X[0:2,0:2], X[2:4, 2:4], X[4:10, 4:10]
X_BD_array = block_diag(A,B,C)
X_BD = Qobj(X_BD_array)
# X_BD = X_BD.tidyup(atol=1e-3)
XP = X_BD * X_BD.conj().trans()

# find T
T = X * X_BD.conj().trans() * XP.sqrtm().inv()

# find H_R_BD
T.dims = H_R.dims
H_R_BD =  T.conj().trans() * H_R * T
H_R_BD

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[-5.03086658e+01+0.j         -8.44459791e-01+7.75840776j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j        ]
 [-8.44459791e-01-7.75840776j -5.55097174e+01+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
  -1.38350872e+01+0.j          7.66132116e-01+0.36148883j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
   7.66132116e-01-0.36148883j -2.06532810e+01+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00

In [199]:
H_R_BD.tidyup(atol=1.5e-4)


Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[-5.03086658e+01+0.j         -8.44459791e-01+7.75840776j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j        ]
 [-8.44459791e-01-7.75840776j -5.55097174e+01+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
  -1.38350872e+01+0.j          7.66132116e-01+0.36148883j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
   7.66132116e-01-0.36148883j -2.06532810e+01+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00

In [200]:
def from16to8(M):
    return np.array([M[0,0],M[1,0],M[0,1],M[1,1],M[2,2],M[3,2],M[2,3],M[3,3]]).reshape(8,1)

# Get Pauli coeff.
II = tensor(qeye(2), qeye(2))
IX = tensor(qeye(2), sigmax())
IY = tensor(qeye(2), sigmay())
IZ = tensor(qeye(2), sigmaz())
ZI = tensor(sigmaz(), qeye(2))
ZY = tensor(sigmaz(), sigmay())
ZX = tensor(sigmaz(), sigmax())
ZZ = tensor(sigmaz(), sigmaz())

a = np.column_stack([from16to8(II),from16to8(IX),from16to8(IY),from16to8(IZ)
                    ,from16to8(ZI),from16to8(ZX),from16to8(ZY),from16to8(ZZ)])

b = from16to8(H_R_BD[0:4, 0:4])

c = np.linalg.solve(a, b)

In [201]:
sympy.Matrix(a)
sympy.Matrix(c)

Matrix([
[                          -35.0766878283766],
[                        -0.0391638374208262],
[  -4.0599482951568 - 8.32667268468867e-16*I],
[                           3.00481132566825],
[                          -17.8325037409416],
[-0.805295953814262 + 8.88178419700125e-16*I],
[ -3.69845946305381 - 1.16573417585641e-15*I],
[                         -0.404285539610635]])

In [202]:
test=Qobj(IX.data.reshape(16,1))
IX.data.reshape(16,1).todense()

matrix([[0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j],
        [0.+0.j],
        [1.+0.j],
        [0.+0.j],
        [0.+0.j],
        [1.+0.j],
        [0.+0.j]])

In [184]:
N =3

b1 = destroy(N)
b2 = destroy(N)
(tensor(b1.dag()*b1, qeye(N)) + tensor(qeye(N), b2.dag()*b2))

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 3. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 4.]]

# Transmon with a Duffing model

In [53]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from qutip import *
import sympy

In [6]:
# parameters for IBM qubit
w = 5 # in GHz
alpha = -0.35 # in GHz

N = 3
b = destroy(N)

H = w1*b.dag()*b + alpha/2.0*b.dag()*b*(b.dag()*b-qeye(N))
H1 = w1*b.dag()*b + alpha/2.0*b.dag()*b.dag()*b*b
Hd = b.dag() + b

In [9]:
H
# Hd

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[ 0.    0.    0.  ]
 [ 0.    5.3   0.  ]
 [ 0.    0.   10.25]]

# Find Pauli-coefficients of CSQF/transmon with CR
Ref: Xuexin's mathematica notebook
1. Build effective two-qubit Hamiltonian.
2. Diagonalize the two-qbuti Hamiltonian to get rid of coupling by either SW or numerical diagonalization. Dressed state.
3. Build driving Hamiltonian in the dressed state.
4. Move into rotating frame.
5. Block-diagonalize the total Hamiltonian by SW or least action.
6. Find Pauli-coefficients.

## Helper

In [14]:
def list_energies(f1,alpha1, beta1, f2, alpha2, beta2, sort=False):
    """
    Calculate energy levels. E00 = 0
    """
    numStates = 4 # 0,1,2,3 states
    state1 = [0, f1, f1+f1+alpha1,f1+f1+f1+alpha1+beta1]
    state2 = [0, f2, f2+f2+alpha2,f2+f2+f2+alpha2+beta2]
    
    result_dict = {}
    for prod in itertools.product(range(numStates), range(numStates)):
        temp = {prod:state1[prod[0]] + state2[prod[1]]}
        result_dict.update(temp)
    
    if sort:
        sorted_dict = OrderedDict(sorted(result_dict.items(), key=lambda x: x[1]))
        return sorted_dict
    else:
        return result_dict
    
list_energies(5.05, 0.6, 0.5, 5.25,-0.33, -0.37,sort=True)    
# list_energies(5.25,-0.33, -0.37, 5.05, -0.3, -0.33, sort=True)    

def sorted_diag_op(eigen):
    """
    Diagonalizing matrix by the order of eigenvector
    
    Args:
        eigen : output of eigenstates() method in qutip
       
    Return: a new diagonalizing operator Qobj
    """
       
    index_list = [np.argmax(np.absolute(vec)) for vec in eigen[1]]
    
    X_array = np.column_stack([eigen[1][index_list.index(i)] for i in range(eigen[1].size)])
    X = Qobj(X_array)
    
    return X

## Define qubit frequencies and J

In [16]:
wm = 5164       # CSFQ w01
del_m0 = 600   # first anharmonicity
del_m1 = 430  # second anharmon.
gdm = 80     # CSFQ-bus coupling

wr = 6292 # bus cavity

wt = 5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = -80    # transmon-bus coupling

gmt = 0  # csfq-transmon direct g

# CSFQ, j=0,1,2
w1_d = lambda j: wm*j + del_m0/2*j*(j-1) if j<3 else wm*j + del_m0/2*j*(j-1) + del_m1-del_m0 # dressed freq
gamma1 = lambda j: j/(wr + w1_d(j-1) - w1_d(j))
w1_b = lambda j: w1_d(j) - gamma1(j) * gdm**2  # bare freq. j=1,2,3

# transmon, j=0,1,2
w2_d = lambda j: wt*j + del_t0/2*j*(j-1) if j<3 else wt*j + del_t0/2*j*(j-1) + del_t1-del_t0 # dressed freq
gamma2 = lambda j: j/(wr + w2_d(j-1) - w2_d(j))
w2_b = lambda j: w2_d(j) - gamma2(j) * gdt**2  # bare freq.

# J, {j1,j2}=0,1,2
J = lambda j1,j2 : gmt - gdm*gdt/2*(1/(wr+w1_d(j1)-w1_d(j1+1)) + 1/(wr+w2_d(j2)-w2_d(j2+1)) 
                                    + 1/(wr-w1_d(j1)+w1_d(j1+1)) + 1/(wr-w2_d(j2)+w2_d(j2+1))) 

In [17]:
J(0,1)
w1_d(1)-w1_d(0)
# w2_d(1)-w2_d(0)
j1=0
j2=1
- gdm*gdt/2*(1/(wr+w1_d(j1)-w1_d(j1+1)) + 1/(wr+w2_d(j2)-w2_d(j2+1)) 
                                    + 1/(wr-w1_d(j1)+w1_d(j1+1)) + 1/(wr-w2_d(j2)+w2_d(j2+1))) 
# J(0,0)

5.808173919146181

## Build effective two-qubit Hamiltonian

In [18]:
def get_possible_states(n):
    """
    Get a list of tuples of possible states, given n
    Ex) for n=2, output is [(0,0),(0,1),(1,0),(0,2),(1,1),(2,0)]
    Args: 
        n: integer such that n1+n2<=n for |n1,n2> state
    Return:
        List of tuples that satisfy n1+n2<=n where n1 and n2 are 0,1,2,3,....
    """
    def get_possible_sum(n):
        """
         [(0,1)]
        """
        result = []
        for i in range(n+1):
            result.append((i,n-i))
        return result
    
    possible_list = []
    for i in range(n + 1):
        possible_list += get_possible_sum(i)
    
    return possible_list
def kronecker_delta(i,j):
    return (1 if i==j else 0)

# b1[i,j] = <i|b1|j>, where |i> = {|00>, |01>, |10>, |11>, ... |30>}
# b1|n1_i,n2_i> = sqrt(n1_i)|n1_i-1,n2_i> where |i> = |n1_i, n2_i> and |j> = |n1_j, n2_j>
# b2|n1_i,n2_i> = sqrt(n2_i)|n1_i,n2_i-1> where |i> = |n1_i, n2_i> and |j> = |n1_j, n2_j>
# V = sqrt((n1+1)*(n2+1))*J_n1_n2*(|n1+1,n2><n1,n2+1|+|n1,n2+1><n1+1,n2|)

states = get_possible_states(3) # up to |3> state
unsorted_state_energy_dict = { s: w1_b(s[0]) + w2_b(s[1]) for s in states}
sorted_state_energy_dict = OrderedDict(sorted(unsorted_state_energy_dict.items(), key=lambda x: x[1]))

sorted_energy = np.array(list(sorted_state_energy_dict.values()))
sorted_states = list(sorted_state_energy_dict.keys())

sorted_states= [(0,0),(0,1),(1,0),(1,1),(0,2),(2,0),(0,3),(1,2),(2,1),(3,0)]
sorted_energy = np.array([0, w2_b(1), w1_b(1), w1_b(1)+w2_b(1), w2_b(2), w1_b(2), w2_b(3)
                 ,w1_b(1)+w2_b(2), w1_b(2)+w2_b(1), w1_b(3)])

n = len(sorted_state_energy_dict)
b1_array, b2_array, V0_array = np.zeros((n,n)), np.zeros((n,n)), np.zeros((n,n))

for i, j in itertools.product(range(n), range(n)):
    n1_i, n1_j = sorted_states[i][0], sorted_states[j][0]
    n2_i, n2_j = sorted_states[i][1], sorted_states[j][1]
    
    # b1_array[i,j] = <n1_i, n2_i|b1|n1_j, n2_j>
    b1_array[i, j] = np.sqrt(n1_j)* kronecker_delta(n1_i, n1_j-1) * kronecker_delta(n2_i, n2_j)
    b2_array[i, j] = np.sqrt(n2_j)* kronecker_delta(n2_i, n2_j-1) * kronecker_delta(n1_i, n1_j)
   
    # 1) n1=n1_i-1, n2 = n2_i,  2) n1=n1_i, n2=n2_i-1
    V0_array[i,j] = (np.sqrt(n1_i*(n2_i+1))*J(n1_i-1, n2_i)*kronecker_delta(n1_i-1, n1_j)*kronecker_delta(n2_i+1, n2_j)
                   + np.sqrt((n1_i+1)*n2_i)*J(n1_i, n2_i-1)*kronecker_delta(n1_i+1, n1_j)*kronecker_delta(n2_i-1, n2_j))

H0 = Qobj(np.diag(sorted_energy))    
V0 = Qobj(V0_array)

b1 = Qobj(b1_array)
b2 = Qobj(b2_array)

b1t = b1 + b1.dag()
b2t = b2 + b2.dag()
ntot = b1.dag()*b1 + b2.dag()*b2

In [19]:
V0

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          6.59245214  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          6.59245214  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          8.21399833 13.8625133
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          8.21399833  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         13.8625133   0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          9.1692497   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   9.1692497  

In [20]:
# states to consider
# 00, 01,10,11,02,20,03,12,21,30
diag = np.array([0, w2_b(1), w1_b(1), w1_b(1)+w2_b(1), w2_b(2), w1_b(2), w2_b(3)
                 ,w1_b(1)+w2_b(2), w1_b(2)+w2_b(1), w1_b(3)])
H0 = Qobj(np.diag(diag))

b1dag = np.zeros((10,10))
b1dag[2,0] = 1
b1dag[3,1] = 1
b1dag[5,2] = np.sqrt(2)
b1dag[8,3] = np.sqrt(2)
b1dag[7,4] = 1
b1dag[9,5] = np.sqrt(3)
b1dag = Qobj(b1dag)

b2dag = np.zeros((10,10))
b2dag[1,0] = 1
b2dag[3,2] = 1
b2dag[4,1] = np.sqrt(2)
b2dag[7,3] = np.sqrt(2)
b2dag[6,4] = np.sqrt(3)
b2dag[8,5] = 1
b2dag = Qobj(b2dag)

V0  = np.zeros((10,10))
V0[2,1] = J(0,0)
V0[1,2] = J(0,0)
V0[4,3] = np.sqrt(2)*J(0,1)
V0[5,3] = np.sqrt(2)*J(1,0)
V0[3,4] = np.sqrt(2)*J(0,1)
V0[3,5] = np.sqrt(2)*J(1,0)
V0[7,6] = np.sqrt(3)*J(0,2)
V0[6,7] = np.sqrt(3)*J(0,2)
V0[8,7] = np.sqrt(4)*J(1,1)
V0[7,8] = np.sqrt(4)*J(1,1)
V0[9,8] = np.sqrt(3)*J(2,0)
V0[8,9] = np.sqrt(3)*J(2,0)
V0 = Qobj(V0)

b1 = b1dag.dag()
b2 = b2dag.dag()

b1t = b1 + b1dag
b2t = b2 + b2dag
ntot = b1dag*b1 + b2dag*b2

In [21]:
V0

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          6.59245214  0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          6.59245214  0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          8.21399833 13.8625133
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          8.21399833  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.         13.8625133   0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          9.1692497   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   9.1692497  

In [23]:
H0+V0

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.28560000e+03 6.59245214e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.59245214e+00 5.15832624e+03 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.04439262e+04
  8.21399833e+00 1.38625133e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 8.21399833e+00
  1.02452694e+04 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.38625133e+01
  0.00000000e+00 1.09037576e+04 0.00000000e+00 0.00000000e+

In [58]:
b1dag

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         1.41421356 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         1.41421356 0. 

## Diagonalize two-qubit Hamiltonian

In [63]:
# By solving eigenvalue
H1 = H0 + V0
eigen = H1.eigenstates()

# find U
U = sorted_diag_op(eigen)
U.dims = H1.dims

# digonalize H0
H2 = U.dag() * H1 * U

In [9]:
U

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  9.98668329e-01 -5.15903950e-02  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  5.15903950e-02  9.98668329e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.98692815e-01
   4.13024295e-02 -3.01126357e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.13098659e-02
  -9.99146313e-01 -3.75387721e-04  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -3.01024333e-02
  -8

In [248]:
sympy.Matrix(np.round(H2,2))

Matrix([
[0,       0,       0,        0,        0,        0,        0,        0,        0,        0],
[0, 5285.94,       0,        0,        0,        0,        0,        0,        0,        0],
[0,       0, 5157.99,        0,        0,        0,        0,        0,        0,        0],
[0,       0,       0, 10443.85,        0,        0,        0,        0,        0,        0],
[0,       0,       0,        0, 10244.93,        0,        0,        0,        0,        0],
[0,       0,       0,        0,        0, 10904.18,        0,        0,        0,        0],
[0,       0,       0,        0,        0,        0, 14836.65,        0,        0,        0],
[0,       0,       0,        0,        0,        0,        0, 15403.33,        0,        0],
[0,       0,       0,        0,        0,        0,        0,        0, 16184.42,        0],
[0,       0,       0,        0,        0,        0,        0,        0,        0, 16931.44]])

## Driving Hamiltonian in dressed state

In [64]:
# get new Hd and HR
Omega = 1 # MHz
wd = (H2[3,3]-H2[2,2]+H2[1,1])/2  # 10443 - 5157 + 5285 in MHz
Hd = lambda t: Omega * np.cos(2*pi*wd*t)* b1t
Hd_D = lambda t: U.dag() * Hd(t) * U

# To rotating frame
R = lambda t: (-1j*2*pi*wd*ntot*t).expm()
HR = lambda t: R(t).dag() * (H2 + Hd_D(t)) * R(t) - ntot *wd

# average over 10 period
t_list = np.linspace(0, 2, 401)*1e-3  # in us
HR_list = [HR(t) for t in t_list]
H_R = functools.reduce(operator.add, HR_list) / len(HR_list)
H_R

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = False
Qobj data =
[[ 0.00000000e+00+0.00000000e+00j  2.59978919e-02-9.82006788e-05j
   5.03257850e-01-1.90093345e-03j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 2.59978919e-02+9.82006788e-05j  3.90391377e-02+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  5.01493236e-01-1.89426805e-03j
   2.07538198e-02-7.83924789e-05j -5.19043159e-02+1.96055860e-04j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
   0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]
 [ 5.03257850e-01+1.90093345e-03j  0.00000000e+00+0.00000000e+00j
  -1.27915841e+02+0.00000000e+00j -4.73882334e-02+1.78997463e-04j
  -1.69229258e-03+6.39222138e-06j -7.10608408e-01+2.68414948e-03j
   0.00000000e+00+0.00000000e+00j  0.00000000

In [209]:
avg_list = []
for N in range(2,401):
    temp = (functools.reduce(operator.add,[HR_list[i] for i in range(N)]) / N)
    temp = np.real(temp[0,1])
    avg_list.append(temp)
    

In [210]:
 plt.plot(avg_list)
    
# f=np.fft.fft(avg_list)
# plt.plot(np.absolute(f))
# plt.yscale('log')

## Least action

In [113]:
# least action
eigen = H_R.eigenstates()

# find X
X = sorted_diag_op(eigen)

# find X_BD and XP
A, B, C = X[0:2,0:2], X[2:4, 2:4], X[4:10, 4:10]
X_BD_array = block_diag(A,B,C)
X_BD = Qobj(X_BD_array)
# X_BD = X_BD.tidyup(atol=1e-3)
XP = X_BD * X_BD.conj().trans()

# find T
T = X * X_BD.dag() * XP.sqrtm().inv()

# find H_R_BD
T.dims = H_R.dims
H_R_BD =  T.dag() * H_R * T
H_R_BD.tidyup(1e-4)

Quantum object: dims = [[10], [10]], shape = (10, 10), type = oper, isherm = True
Qobj data =
[[ 1.97993647e-03+0.j          2.59963234e-02+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 2.59963234e-02+0.j          4.09972254e-02+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
  -1.27918918e+02+0.j         -4.73867614e-02+0.00017899j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j
   0.00000000e+00+0.j          0.00000000e+00+0.j        ]
 [ 0.00000000e+00+0.j          0.00000000e+00+0.j
  -4.73867614e-02-0.00017899j -1.27957960e+02+0.j
   0.00000000e+00+0.j

## Pauli coefficient

In [66]:
def from16to8(M):
    return np.array([M[0,0],M[1,0],M[0,1],M[1,1],M[2,2],M[3,2],M[2,3],M[3,3]]).reshape(8,1)

# Get Pauli coeff.
II = tensor(qeye(2), qeye(2))
IX = tensor(qeye(2), sigmax())
IY = tensor(qeye(2), sigmay())
IZ = tensor(qeye(2), sigmaz())
ZI = tensor(sigmaz(), qeye(2))
ZY = tensor(sigmaz(), sigmay())
ZX = tensor(sigmaz(), sigmax())
ZZ = tensor(sigmaz(), sigmaz())

a = np.column_stack([from16to8(II),from16to8(IX)/2,from16to8(IY)/2,from16to8(IZ)/2
                    ,from16to8(ZI)/2,from16to8(ZX)/2,from16to8(ZY)/2,from16to8(ZZ)/2])

b = from16to8(H_R_BD[0:4, 0:4])

c = np.linalg.solve(a, b)
sympy.Matrix(np.round(c,6))

Matrix([
[-63.958475],
[  -0.02139],
[   -8.1e-5],
[    1.2e-5],
[127.959928],
[  0.073383],
[  0.000277],
[  -0.03903]])

## Pauli vs CR amplitude

In [67]:
def get_Pauli_coeff(Omega):
    # get new Hd and HR
#     Omega = 1 # MHz
    wd = (H2[3,3]-H2[2,2]+H2[1,1])/2  # 10443 - 5157 + 5285 in MHz
    Hd = lambda t: Omega * np.cos(2*pi*wd*t)* b1t
    Hd_D = lambda t: U.dag() * Hd(t) * U

    # To rotating frame
    R = lambda t: (-1j*2*pi*wd*ntot*t).expm()
    HR = lambda t: R(t).dag() * (H2 + Hd_D(t)) * R(t) - ntot *wd

    # average over 10 period
    t_list = np.linspace(0, 2, 401)*1e-3  # in us
    HR_list = [HR(t) for t in t_list]
    H_R = functools.reduce(operator.add, HR_list) / len(HR_list)
    
    # least action
    eigen = H_R.eigenstates()

    # find X
    X = sorted_diag_op(eigen)

    # find X_BD and XP
    A, B, C = X[0:2,0:2], X[2:4, 2:4], X[4:10, 4:10]
    X_BD_array = block_diag(A,B,C)
    X_BD = Qobj(X_BD_array)
    # X_BD = X_BD.tidyup(atol=1e-3)
    XP = X_BD * X_BD.conj().trans()

    # find T
    T = X * X_BD.dag() * XP.sqrtm().inv()

    # find H_R_BD
    T.dims = H_R.dims
    H_R_BD =  T.dag() * H_R * T
   
    def from16to8(M):
        return np.array([M[0,0],M[1,0],M[0,1],M[1,1],M[2,2],M[3,2],M[2,3],M[3,3]]).reshape(8,1)

    # Get Pauli coeff.
    II = tensor(qeye(2), qeye(2))
    IX = tensor(qeye(2), sigmax())
    IY = tensor(qeye(2), sigmay())
    IZ = tensor(qeye(2), sigmaz())
    ZI = tensor(sigmaz(), qeye(2))
    ZY = tensor(sigmaz(), sigmay())
    ZX = tensor(sigmaz(), sigmax())
    ZZ = tensor(sigmaz(), sigmaz())

    a = np.column_stack([from16to8(II),from16to8(IX)/2,from16to8(IY)/2,from16to8(IZ)/2
                        ,from16to8(ZI)/2,from16to8(ZX)/2,from16to8(ZY)/2,from16to8(ZZ)/2])

    b = from16to8(H_R_BD[0:4, 0:4])

    c = np.linalg.solve(a, b)
    
    return c

In [68]:
# plot Pauli coeff. vs CR amplitude
Omega_list = np.linspace(1, 200, 10)
Pauli = get_Pauli_coeff(Omega_list[0])
for omega in Omega_list[1:]:
    Pauli = np.column_stack((Pauli, get_Pauli_coeff(omega)))

In [69]:
# plot
Pauli_label = ['II','IX','IY','IZ','ZI','ZX','ZY','ZZ']
fig, ax = plt.subplots(1,1, figsize=(10,8))
for i in range(Pauli.shape[0]):
    ax.plot(Omega_list, Pauli[i], label=Pauli_label[i], linewidth=3)
ax.set_xlabel('CR amplitude, Omega (MHz)', fontsize=18)
ax.set_ylabel('$Pauli Coefficient$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

/usr/lib/python3/dist-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [273]:
# ZZ

fig, ax = plt.subplots(1,1, figsize=(10,8))
i = 7
ax.plot(Omega_list, Pauli[i], label=Pauli_label[i], linewidth=3)
ax.set_xlabel('CR amplitude, Omega (MHz)', fontsize=18)
ax.set_ylabel(r'$\zeta$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

# Function : get_Pauli_coefficient

In [116]:
def sorted_diag_op(eigen):
    """
    Diagonalizing matrix by the order of eigenvector
    
    Args:
        eigen : output of eigenstates() method in qutip
       
    Return: a new diagonalizing operator Qobj
    """
       
    index_list = [np.argmax(np.absolute(vec)) for vec in eigen[1]]
    
    X_array = np.column_stack([eigen[1][index_list.index(i)] for i in range(eigen[1].size)])
    X = Qobj(X_array)
    
    return X

def get_Pauli_coefficient(wm,del_m0,del_m1,gdm
                         ,wt,del_t0,del_t1,gdt
                         ,wr,gmt,Omega):

    # CSFQ, j=0,1,2
    w1_d = lambda j: wm*j + del_m0/2*j*(j-1) if j<3 else wm*j + del_m0/2*j*(j-1) + del_m1-del_m0 # dressed freq
    gamma1 = lambda j: j/(wr + w1_d(j-1) - w1_d(j))
    w1_b = lambda j: w1_d(j) - gamma1(j) * gdm**2  # bare freq. j=1,2,3

    # transmon, j=0,1,2
    w2_d = lambda j: wt*j + del_t0/2*j*(j-1) if j<3 else wt*j + del_t0/2*j*(j-1) + del_t1-del_t0 # dressed freq
    gamma2 = lambda j: j/(wr + w2_d(j-1) - w2_d(j))
    w2_b = lambda j: w2_d(j) - gamma2(j) * gdt**2  # bare freq.

    # J, {j1,j2}=0,1,2
    J = lambda j1,j2 : gmt - gdm*gdt/2*(1/(wr+w1_d(j1)-w1_d(j1+1)) + 1/(wr+w2_d(j2)-w2_d(j2+1)) 
                                        + 1/(wr-w1_d(j1)+w1_d(j1+1)) + 1/(wr-w2_d(j2)+w2_d(j2+1))) 
    
    def get_possible_states(n):
        """
        Get a list of tuples of possible states, given n
        Ex) for n=2, output is [(0,0),(0,1),(1,0),(0,2),(1,1),(2,0)]
        Args: 
            n: integer such that n1+n2<=n for |n1,n2> state
        Return:
            List of tuples that satisfy n1+n2<=n where n1 and n2 are 0,1,2,3,....
        """
        def get_possible_sum(n):
                """
                 [(0,1)]
                """
                result = []
                for i in range(n+1):
                    result.append((i,n-i))
                return result

        possible_list = []
        for i in range(n + 1):
            possible_list += get_possible_sum(i)

        return possible_list
    def kronecker_delta(i,j):
        return (1 if i==j else 0)

    # Get H0, V0, b1 and b2
    states = get_possible_states(3) # up to |3> state
    unsorted_state_energy_dict = { s: w1_b(s[0]) + w2_b(s[1]) for s in states}
   
    sorted_state_energy_dict = OrderedDict(sorted(unsorted_state_energy_dict.items(), key=lambda x: x[1]))
    sorted_energy = np.array(list(sorted_state_energy_dict.values()))
    sorted_states = list(sorted_state_energy_dict.keys())

#     Xuexin
    sorted_states= [(0,0),(0,1),(1,0),(1,1),(0,2),(2,0),(0,3),(1,2),(2,1),(3,0)]
    sorted_energy = np.array([0, w2_b(1), w1_b(1), w1_b(1)+w2_b(1), w2_b(2), w1_b(2), w2_b(3)
                 ,w1_b(1)+w2_b(2), w1_b(2)+w2_b(1), w1_b(3)])

    n = len(sorted_state_energy_dict)
    b1_array, b2_array, V0_array = np.zeros((n,n)), np.zeros((n,n)), np.zeros((n,n))

    for i, j in itertools.product(range(n), range(n)):
        n1_i, n1_j = sorted_states[i][0], sorted_states[j][0]
        n2_i, n2_j = sorted_states[i][1], sorted_states[j][1]

        # b1_array[i,j] = <n1_i, n2_i|b1|n1_j, n2_j>
        b1_array[i, j] = np.sqrt(n1_j)* kronecker_delta(n1_i, n1_j-1) * kronecker_delta(n2_i, n2_j)
        b2_array[i, j] = np.sqrt(n2_j)* kronecker_delta(n2_i, n2_j-1) * kronecker_delta(n1_i, n1_j)

        # 1) n1=n1_i-1, n2 = n2_i,  2) n1=n1_i, n2=n2_i-1
        V0_array[i,j] = (np.sqrt(n1_i*(n2_i+1))*J(n1_i-1, n2_i)*kronecker_delta(n1_i-1, n1_j)*kronecker_delta(n2_i+1, n2_j)
                       + np.sqrt((n1_i+1)*n2_i)*J(n1_i, n2_i-1)*kronecker_delta(n1_i+1, n1_j)*kronecker_delta(n2_i-1, n2_j))

    H0 = Qobj(np.diag(sorted_energy))    
    V0 = Qobj(V0_array)

    b1 = Qobj(b1_array)
    b2 = Qobj(b2_array)
    
    b1t = b1 + b1.dag()
    b2t = b2 + b2.dag()
    ntot = b1.dag()*b1 + b2.dag()*b2

    # By solving eigenvalue
    H1 = H0 + V0
    eigen = H1.eigenstates()

    # find U
    U = sorted_diag_op(eigen)
    U.dims = H1.dims

    # digonalize H0
    H2 = U.dag() * H1 * U
    
    # get new Hd and HR
#     Omega = 1 # MHz
    wd = (H2[3,3]-H2[2,2]+H2[1,1])/2  # 10443 - 5157 + 5285 in MHz
    Hd = lambda t: Omega * np.cos(2*pi*wd*t)* b1t
    Hd_D = lambda t: U.dag() * Hd(t) * U

    # To rotating frame
    R = lambda t: (-1j*2*pi*wd*ntot*t).expm()
    HR = lambda t: R(t).dag() * (H2 + Hd_D(t)) * R(t) - ntot *wd

    # average over 10 period
    t_list = np.linspace(0, 2, 401)*1e-3  # in us
    HR_list = [HR(t) for t in t_list]
    H_R = functools.reduce(operator.add, HR_list) / len(HR_list)
        
    # least action
    eigen = H_R.eigenstates()

    # find X
    X = sorted_diag_op(eigen)

    # find X_BD and XP
    A, B, C = X[0:2,0:2], X[2:4, 2:4], X[4:10, 4:10]
    X_BD_array = block_diag(A,B,C)
    X_BD = Qobj(X_BD_array)
    # X_BD = X_BD.tidyup(atol=1e-3)
    XP = X_BD * X_BD.dag()

    # find T
    T = X * X_BD.dag() * XP.sqrtm().inv()

    # find H_R_BD
    T.dims = H_R.dims
    H_R_BD =  T.dag() * H_R * T
       
    def from16to8(M):
        return np.array([M[0,0],M[1,0],M[0,1],M[1,1],M[2,2],M[3,2],M[2,3],M[3,3]]).reshape(8,1)

    # Get Pauli coeff.
    II = tensor(qeye(2), qeye(2))
    IX = tensor(qeye(2), sigmax())
    IY = tensor(qeye(2), sigmay())
    IZ = tensor(qeye(2), sigmaz())
    ZI = tensor(sigmaz(), qeye(2))
    ZY = tensor(sigmaz(), sigmay())
    ZX = tensor(sigmaz(), sigmax())
    ZZ = tensor(sigmaz(), sigmaz())

    a = np.column_stack([from16to8(II),from16to8(IX)/2,from16to8(IY)/2,from16to8(IZ)/2
                        ,from16to8(ZI)/2,from16to8(ZX)/2,from16to8(ZY)/2,from16to8(ZZ)/2])

    b = from16to8(H_R_BD[0:4, 0:4])

    c = np.linalg.solve(a, b)
    
    return c

In [119]:
# parameters CSFQ-transmon
wm = 5164       # CSFQ w01
del_m0 = 600   # first anharmonicity
del_m1 = 430  # second anharmon.
gdm = 80     # CSFQ-bus coupling

wr = 6292 # bus cavity

wt = 5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = 80    # transmon-bus coupling

gmt = 0  # csfq-transmon direct g

In [124]:
# parameters, transmon-transmon
wm = 5292 #5164       # CSFQ w01
del_m0 = -330 #600   # first anharmonicity
del_m1 = -370 #430  # second anharmon.
gdm = 80     # CSFQ-bus coupling

wr = 6292 # bus cavity

wt = 5164 #5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = 80    # transmon-bus coupling

gmt = 0  # csfq-transmon direct g

In [54]:
# parameters, transmon-transmon (usual target-control swapped)
wm = 5164       # CSFQ w01
del_m0 = -330 #600   # first anharmonicity
del_m1 = -370 #430  # second anharmon.
gdm = 80    # CSFQ-bus coupling

wr = 6292 # bus cavity

wt = 5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = -80   # transmon-bus coupling

gmt = 2.7  # csfq-transmon direct g

In [125]:
# plot Pauli coeff. vs CR amplitude
Omega_list = np.linspace(0, 200, 20)

Pauli = get_Pauli_coefficient(wm,del_m0,del_m1,gdm
                         ,wt,del_t0,del_t1,gdt
                         ,wr,gmt,Omega_list[0])
for omega in Omega_list[1:]:
    Pauli = np.column_stack((Pauli, get_Pauli_coefficient(wm,del_m0,del_m1,gdm
                         ,wt,del_t0,del_t1,gdt,wr,gmt,omega)))
    
# plot
Pauli_label = ['II','IX','IY','IZ','ZI','ZX','ZY','ZZ']
fig, ax = plt.subplots(1,1, figsize=(10,8))
for i in range(Pauli.shape[0]):
    ax.plot(Omega_list, Pauli[i], label=Pauli_label[i], linewidth=3)
ax.set_xlabel('CR amplitude, Omega (MHz)', fontsize=18)
ax.set_ylabel('$Pauli Coefficient$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

/usr/lib/python3/dist-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


# Function : get ZZ

In [6]:
def get_static_ZZ(wm,del_m0,del_m1,gdm
                  ,wt,del_t0,del_t1,gdt
                         ,wr,gmt):
    """
    By diagonalizing two-qubit Hamiltonian, calculate ZZ(=E11-E01-E10+E00).
    """
    
    # CSFQ, j=0,1,2
    w1_d = lambda j: wm*j + del_m0/2*j*(j-1) if j<3 else wm*j + del_m0/2*j*(j-1) + del_m1-del_m0 # dressed freq
    gamma1 = lambda j: j/(wr + w1_d(j-1) - w1_d(j))
    w1_b = lambda j: w1_d(j) - gamma1(j) * gdm**2  # bare freq. j=1,2,3

    # transmon, j=0,1,2
    w2_d = lambda j: wt*j + del_t0/2*j*(j-1) if j<3 else wt*j + del_t0/2*j*(j-1) + del_t1-del_t0 # dressed freq
    gamma2 = lambda j: j/(wr + w2_d(j-1) - w2_d(j))
    w2_b = lambda j: w2_d(j) - gamma2(j) * gdt**2  # bare freq.

    # J, {j1,j2}=0,1,2
    J = lambda j1,j2 : gmt - gdm*gdt/2*(1/(wr+w1_d(j1)-w1_d(j1+1)) + 1/(wr+w2_d(j2)-w2_d(j2+1)) 
                                        + 1/(wr-w1_d(j1)+w1_d(j1+1)) + 1/(wr-w2_d(j2)+w2_d(j2+1))) 
    
    def get_possible_states(n):
        """
        Get a list of tuples of possible states, given n
        Ex) for n=2, output is [(0,0),(0,1),(1,0),(0,2),(1,1),(2,0)]
        Args: 
            n: integer such that n1+n2<=n for |n1,n2> state
        Return:
            List of tuples that satisfy n1+n2<=n where n1 and n2 are 0,1,2,3,....
        """
        def get_possible_sum(n):
                """
                 [(0,1)]
                """
                result = []
                for i in range(n+1):
                    result.append((i,n-i))
                return result

        possible_list = []
        for i in range(n + 1):
            possible_list += get_possible_sum(i)

        return possible_list
    def kronecker_delta(i,j):
        return (1 if i==j else 0)   
    def sorted_diag_op(eigen):
        """
        Diagonalizing matrix by the order of eigenvector

        Args:
            eigen : output of eigenstates() method in qutip

        Return: a new diagonalizing operator Qobj
        """

        index_list = [np.argmax(np.absolute(vec)) for vec in eigen[1]]

        X_array = np.column_stack([eigen[1][index_list.index(i)] for i in range(eigen[1].size)])
        X = Qobj(X_array)

        return X
    
    # Get H0, V0, b1 and b2
    states = get_possible_states(3) # up to |3> state
    unsorted_state_energy_dict = { s: w1_b(s[0]) + w2_b(s[1]) for s in states}
   
    sorted_state_energy_dict = OrderedDict(sorted(unsorted_state_energy_dict.items(), key=lambda x: x[1]))
    sorted_energy = np.array(list(sorted_state_energy_dict.values()))
    sorted_states = list(sorted_state_energy_dict.keys())

#     Xuexin
    sorted_states= [(0,0),(0,1),(1,0),(1,1),(0,2),(2,0),(0,3),(1,2),(2,1),(3,0)]
    sorted_energy = np.array([0, w2_b(1), w1_b(1), w1_b(1)+w2_b(1), w2_b(2), w1_b(2), w2_b(3)
                 ,w1_b(1)+w2_b(2), w1_b(2)+w2_b(1), w1_b(3)])

    n = len(sorted_state_energy_dict)
    b1_array, b2_array, V0_array = np.zeros((n,n)), np.zeros((n,n)), np.zeros((n,n))

    for i, j in itertools.product(range(n), range(n)):
        n1_i, n1_j = sorted_states[i][0], sorted_states[j][0]
        n2_i, n2_j = sorted_states[i][1], sorted_states[j][1]

        # b1_array[i,j] = <n1_i, n2_i|b1|n1_j, n2_j>
        b1_array[i, j] = np.sqrt(n1_j)* kronecker_delta(n1_i, n1_j-1) * kronecker_delta(n2_i, n2_j)
        b2_array[i, j] = np.sqrt(n2_j)* kronecker_delta(n2_i, n2_j-1) * kronecker_delta(n1_i, n1_j)

        # 1) n1=n1_i-1, n2 = n2_i,  2) n1=n1_i, n2=n2_i-1
        V0_array[i,j] = (np.sqrt(n1_i*(n2_i+1))*J(n1_i-1, n2_i)*kronecker_delta(n1_i-1, n1_j)*kronecker_delta(n2_i+1, n2_j)
                       + np.sqrt((n1_i+1)*n2_i)*J(n1_i, n2_i-1)*kronecker_delta(n1_i+1, n1_j)*kronecker_delta(n2_i-1, n2_j))

    H0 = Qobj(np.diag(sorted_energy))    
    V0 = Qobj(V0_array)

    b1 = Qobj(b1_array)
    b2 = Qobj(b2_array)
    
    b1t = b1 + b1.dag()
    b2t = b2 + b2.dag()
    ntot = b1.dag()*b1 + b2.dag()*b2

    # By solving eigenvalue
    H1 = H0 + V0
    eigen = H1.eigenstates()

    # find U
    U = sorted_diag_op(eigen)
    U.dims = H1.dims

    # digonalize H0
    H2 = U.dag() * H1 * U
    
    return (H2[3,3]-H2[2,2]-H2[1,1], J(0,0))

In [17]:
# parameters, transmon-transmon
wm = 5292 #5164       # transmon
del_m0 = -330 #600   # first anharmonicity
del_m1 = -370 #430  # second anharmon.
gdm = 120     # transmon-bus coupling

wr = 6292 # bus cavity

wt = 5164 #5292   # transmon
del_t0 = -329.1
del_t1 = -369.7
gdt = 120    # transmon-bus coupling

gmt = 3  # csfq-transmon direct g

In [18]:
get_static_ZZ(wm,del_m0,del_m1,gdm
              ,wt,del_t0,del_t1,gdt
              ,wr,gmt)[1]

-11.833017304841954

In [19]:
# plot Pauli coeff. vs CR amplitude
wr_list = np.linspace(5500, 8000, 101)

ZZ = [get_static_ZZ(wm,del_m0,del_m1,gdm
                   ,wt,del_t0,del_t1,gdt
                   ,wr,gmt)[0] for wr in wr_list]
J = [get_static_ZZ(wm,del_m0,del_m1,gdm
                   ,wt,del_t0,del_t1,gdt
                   ,wr,gmt)[1] for wr in wr_list]  
# plot
fig, ax = plt.subplots(1,1, figsize=(10,8))
ax.plot(wr_list, ZZ, linewidth=3, label='ZZ')
ax.plot(wr_list, J, linewidth=3, label='J')
ax.set_xlabel('wr (MHz)', fontsize=18)
ax.set_ylabel('$ZZ$ (MHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

/usr/lib/python3/dist-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [100]:
plt.plot(wr_list, J)

# Test

In [132]:
from sympy import I
Omega = 10
wd = sympy.Symbol('wd')
t = sympy.Symbol('t')

Hd = Omega * (sympy.exp(-I*wd*t) + sympy.exp(I*wd*t))/2 * sympy.Matrix(b1t.data.toarray())
Hdt = (U.dag()).data.toarray() * Hd * U.data.toarray()

Htot = H2.data.toarray() + Hdt

Ur = sympy.exp(-I*wd *sympy.Matrix(ntot.data.toarray())*t)
Urdag = sympy.exp(I*wd *sympy.Matrix(ntot.data.toarray())*t)
H3t = Urdag * Htot * Ur -  sympy.Matrix(ntot.data.toarray())

H3t_RWA = sympy.simplify(H3t).subs(sympy.exp(-2*I*wd*t),0)

In [133]:
sympy.simplify(H3t).subs([(sympy.exp(2*I*wd*t),0), (sympy.oo,0)])


Matrix([
[                0,                 zoo,                zoo,                  0,                    0,                    0,                0,                0,                0,                0],
[ 4.99334164448514,    5156.98568041008,                  0,                zoo,                  zoo,                  zoo,                0,                0,                0,                0],
[0.257951975085346,                   0,   5284.94056072468,                zoo,                  zoo,                  zoo,                0,                0,                0,                0],
[                0, -0.0167909850402803,  0.205920111856128,    10242.929874963,                    0,                    0,              zoo,              zoo,              zoo,              zoo],
[                0,  -0.470187676635091,   4.97583307954979,                  0,     10441.8481628593,                    0,              zoo,              zoo,              zoo,              zoo],
[